## Scrapping Notebook   
This notebook is used to scrap the data from the website. The data is then saved in a csv file. The data is then used in the main notebook for analysis.</br>
This data is scrapped from the website: https://ineedlearn.wordpress.com/2020/12/26/width-height-and-dpr-of-cell-phones/.</br>
This data is then used to create a dataset which is used in the main notebook for analysis.</br>
This website contains the data of the width, height and dpr of the cell phones. This data is scrapped and saved in a csv file.</br>
This data will be used to predict the height of the cell phone based on the width of the the cell phone.</br>



In [1]:
# intsall the following packages 
!pip install selenium

In [2]:
# import the following packages

import pandas as pd
import numpy as np
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service


In [4]:
# to stop the live web scraping and work in the background
chrome_options = Options()
chrome_options.add_argument("--headless")
driver = webdriver.Chrome(options = chrome_options)

url = "https://ineedlearn.wordpress.com/2020/12/26/width-height-and-dpr-of-cell-phones/"

# Open the target URL
driver.get(url)

# the list to store the data 
mobile_prop = []

# Allow some time for the page to load
time.sleep(3)

# Scrape the data the devices are 300 rows in the table
for i in range(1, 300):
    try:
        # Locate the row using XPath
        row = driver.find_element(By.XPATH, f"/html/body/div[1]/div/div/main/article/div[1]/div[1]/figure/table/tbody/tr[{i}]")
        
        # Extract the text content of the row
        mobile_prop.append(row.text)
    except Exception as e:
        # If an error occurs, print the error and continue
        print(f"Error on row {i}: {e}")
        break

# Close the WebDriver
driver.quit()

In [7]:
mobile_prop[:10]

['ALCATEL',
 'Width (px) Height (px) DPR',
 'Alcatel One Touch Idol Ultra 360 316 2',
 'Alcatel One Touch Ultra 995 320 217 1.5',
 'KINDLE FIRE',
 'Width (px) Height (px) DPR',
 'Amazon Kindle Fire 600 1024 1',
 'Amazon Kindle Fire HD 7 533 853 1.5',
 'Amazon Kindle Fire HD 8.9 800 1280 1.5',
 'APLLE']

### Processing the scrapped data 

In [8]:

# Create the DataFrame with the appropriate column names
screen_properties = pd.DataFrame(columns=["Mobile Name", "Width", "Height", "DPR"])

# Populate the DataFrame
for i in range(len(mobile_prop)):
    text = mobile_prop[i].split()
    if len(text) >= 4 and text[-2].replace('.', '').isdigit() and text[-3].replace('.', '').isdigit():
        try:
            # Extract the data from the text
            dpr = float(text[-1]) if text[-1].replace('.', '').isdigit() and float(text[-1]) < 5 else None
            height = float(text[-2])
            width = float(text[-3])
            mobile_name = " ".join(text[:-3])
            
            # Create a new row as a DataFrame
            new_row = pd.DataFrame({
                "Mobile Name": [mobile_name],
                "Width": [width],
                "Height": [height],
                "DPR": [dpr]
            })

            # Append the new row to the DataFrame if it is not empty or all NA
            if not new_row.isna().all(axis=None):
                screen_properties = pd.concat([screen_properties, new_row], ignore_index=True)
        except Exception as e:
            print(f"Error in processing row {i}: {e}")




C:\Users\AHMED OSAMA\AppData\Local\Temp\ipykernel_28176\3978398701.py:25: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  screen_properties = pd.concat([screen_properties, new_row], ignore_index=True)
C:\Users\AHMED OSAMA\AppData\Local\Temp\ipykernel_28176\3978398701.py:25: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  screen_properties = pd.concat([screen_properties, new_row], ignore_index=True)
C:\Users\AHMED OSAMA\AppData\Local\Temp\ipykernel_28176\3978398701.py:25: FutureWarning: The behavior 

In [10]:
# process the dataframe 

screen_properties.isnull().sum()    

Mobile Name    0
Width          0
Height         0
DPR            5
dtype: int64

In [11]:
# view the null values
screen_properties[screen_properties.isnull().any(axis=1)]

,Mobile Name,Width,Height,DPR
28,"Apple Watch Series 1, 3 (38mm)",272.0,340.0,NaN
29,"Apple Watch Series 1, 3 (42mm)",312.0,390.0,NaN
30,Apple Watch Series 5 (40mm),324.0,394.0,NaN
31,Apple Watch Series 5 (44mm),368.0,448.0,NaN
59,Google Pixel,2.0,412.0,NaN


In [13]:
# drop the null values  
screen_properties = screen_properties.dropna()

# Save the DataFrame to a CSV file
screen_properties.to_csv("Source Data/screen_properties.csv", index=False)

print("The data has been saved to 'screen_properties.csv'")

The data has been saved to 'screen_properties.csv'
